# FULL FOLD NEW BASELINE

In [ ]:
CNF={
#     'isTrain':False,
    'plotCAM':False,
    'isTrain':True,
#     'targetFold':[0],
    'targetFold':[0,1,2,3,4],
    'validateSequence':True,
    'batch_size':8,
    #'epoch':30,
    'epoch':10,#10,30
    'lr':0.001,
    'n_tta':3, # unimplemented
    'EMA':True,
    # 'use_fp':True,
    'use_fp':False,
    'nhead':4,#4  8はいまいちな予感
    'nmel':128, # 128 or 256
    'skf_random_state':0, # fold0が比較的バランスが良くなる
    'num_workers':2,
    'loadPthPath':f"../input/pthresnet1229/resnet",
    'backbone':'resnest50', # 'resnest50' or'resnest101' or 'resnet34'
#     'mixup':False,
    'mixup':True,
    'multiLabel':True,
#     'multiLabel':False,
    'trainCUT':6, # 先生は8
    #'trainCUT':5,
    'vgg_sub_sys':True,
    #'pseudo':True,
    'pseudo':False,
    
}


CNF['meltrainTpPath']=f"../input/mymelspec/melspec_tp_NMEL{CNF['nmel']}_FMIN20_FMAX16000_SR32000.npy"
CNF['meltrainFpPath']=f"../input/mymelspec/melspec_fp_NMEL{CNF['nmel']}_FMIN20_FMAX16000_SR32000.npy"
CNF['meltestPath']   =f"../input/mymelspec/melspec_test_NMEL{CNF['nmel']}_FMIN20_FMAX16000_SR32000.npy"

LABEL_COLS = [f"s{_i}" for _i in range(24)]


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
import time
from scipy.interpolate import interp1d
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from scipy.stats import rankdata

from sklearn.linear_model import LogisticRegression


import soundfile as sf
# Librosa Libraries
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
try:
    import cuml as cm
    import cupy as cp
except ModuleNotFoundError:
    pass


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torchvision import datasets, models, transforms
from transformers import *
from math import floor, ceil

from rfcxutil import lwlrap,rfcx_appendFold,rfcx_showSplit,rfcx_foldSplitSanityCheck,rfcx_cut_melspec,rfcx_SimpleDataset
from osicutil import ModelEMA
import pickle
import os,gc,random,glob 

import albumentations as A
from albumentations.core.transforms_interface import BasicTransform


#from resnest.torch import resnest50
#from resnest.torch import resnest101

def seed_everything(seed: int):
    random.seed(seed);os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed);torch.manual_seed(seed);torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

import pickle
def _save(x,fname):
    pickle.dump(x, open(fname, 'wb'))
def _load(fname):
    return pickle.load(open(fname, 'rb'))



In [ ]:
traint = pd.read_csv( '../input/rfcx-species-audio-detection/train_tp.csv' )
trainf = pd.read_csv( '../input/rfcx-species-audio-detection/train_fp.csv' )
traint.shape, trainf.shape

In [ ]:
def set_unique_id(df): # melspec読み込みのためのid
    
    if 'mymelid' in df.columns:
        print(f"WARNING mymelid already in columns")
        df = df.drop(columns='mymelid')
    
    udf = df.drop_duplicates(subset='recording_id', keep='first', ignore_index=True).copy()
    udf['mymelid'] = udf.index.tolist()
    
    return df.merge(udf[['recording_id','mymelid']],how='left',on='recording_id')
    
traint = set_unique_id(traint)
trainf = set_unique_id(trainf)
traint['tp_or_fp'] = 'tp'
trainf['tp_or_fp'] = 'fp'

# Append fold

In [ ]:
traint = rfcx_appendFold(traint,random_state=CNF['skf_random_state'])
rfcx_showSplit(traint)
print(f"Sanity Check [ same recording id -> same fold ] : {rfcx_foldSplitSanityCheck(traint)}")

## CNF['trainCUT']秒間での鳥の種類を記録

In [ ]:
traint['t_center'] = (traint.t_min+traint.t_max)/2
traint['t_start'] = traint['t_center'] - CNF['trainCUT']/2
traint['t_end']   = traint['t_center'] + CNF['trainCUT']/2

# Labeling

In [ ]:
def label_keepindex(_df): # keepindexはdf applyしたときにindexはそのままの意味
    
    ret_df = pd.DataFrame({ f"s{_spid}":  (_df.species_id == _spid).astype(float)  for _spid in range(24)},index=_df.index)
    
    species_ids = [[]]*len(_df)
    
    for idx0,row0 in _df.iterrows():
        
        for idx1,row1 in _df.iterrows():
            
            # 鳴いてる時間row0とrow1に重複があればrow1にrow0.species_idを記録する
            
            if row0.t_end < row1.t_start: # | row0 | | row1 | 
                pass
            elif row1.t_end < row0.t_start: # | row1 | | row0 | 
                pass
            else: # | ro|w r|ow| 
    
                if idx0==idx1:
                    ret_df.loc[idx1,f"s{row0.species_id}"] = 1
                else:
                    if CNF['multiLabel']:
                        ret_df.loc[idx1,f"s{row0.species_id}"] = 1
            
    return ret_df        
    
renew_species_df = traint.groupby('recording_id').apply(label_keepindex)

traint = pd.concat([traint,renew_species_df],axis=1)
traint

## traintをレコード毎にマルチラベル化したtrainu

In [ ]:
# train tp -> レコード事のマルチラベル化 -> train u 
trainu = pd.read_csv( '../input/rfcx-species-audio-detection/train_tp.csv' );trainu = set_unique_id(trainu)
trainu = rfcx_appendFold(trainu,random_state=CNF['skf_random_state'])
def multi_label_keepindex(_df): # keepindexはdf applyしたときにindexはそのままの意味

    ret_df = pd.DataFrame({ f"s{_spid}":  (_df.species_id == _spid).astype(float)  for _spid in range(24)},index=_df.index)
    species_ids = [[]]*len(_df)
    for idx0,row0 in _df.iterrows():
        for idx1,row1 in _df.iterrows():
            ret_df.loc[idx1,f"s{row0.species_id}"] = 1

    return ret_df        

renew_species_df = trainu.groupby('recording_id').apply(multi_label_keepindex)
trainu = pd.concat([trainu,renew_species_df],axis=1)
trainu = trainu.drop_duplicates(subset='recording_id', keep='first', ignore_index=True).copy()

- CUT8 だと158箇所が重複
- CUT6 だと73箇所が重複
- CUT5 だと66箇所が重複


In [ ]:
plt.figure(figsize=(10,2));(traint.t_max-traint.t_min).hist(bins=100);plt.xlabel('sec')
plt.title(f"Song Dulation MIN{(traint.t_max-traint.t_min).min():.3f}sec MAX{(traint.t_max-traint.t_min).max():.3f}sec");

8 秒を切り出してメモリ上に置いておく <strong> traintのindex順に格納するから,　以降のindex操作は慎重にな

In [ ]:
orig_meldata_tp = np.load(CNF['meltrainTpPath'])

#
# メル毎に正規化
#

for idx in range(len(orig_meldata_tp)):
    
    orig_meldata_tp[idx] = ( orig_meldata_tp[idx] - orig_meldata_tp[idx].mean() )/orig_meldata_tp[idx].std()
    orig_meldata_tp[idx] = orig_meldata_tp[idx] - orig_meldata_tp[idx].min() 
    


if True:

    import skimage.transform
    from scipy.interpolate import interp1d

    def f2melf(f):

        _f = np.linspace(20, 32000//2, 128) # min,max,nmel
        _fmel = librosa.core.mel_frequencies(n_mels=128,  fmin = 20,fmax = 32000//2, htk=False)
        return interp1d(_fmel,_f)(f)

    for ridx in ['0201197ec','009b760e6','0099c367b','01b41f92b','0313e82cf','03b96f209','011f25080','0295e3234']:

        idx = traint[traint.recording_id==ridx].mymelid.values[0]

        fig, ax = plt.subplots(figsize=(15, 5))

        img = librosa.display.specshow(
            #melspec,
            orig_meldata_tp[idx], #data
            sr=32000,
            x_axis='time', 
            y_axis='linear', 
            fmin = 20,
            fmax = 32000 // 2,
            ax=ax)


        t_min = traint.iloc[idx].t_min
        t_max = traint.iloc[idx].t_max
        t_ctr = (t_min+t_max)/2

        plt.axvline(t_min,c='w')
        plt.axvline(t_max,c='w')

        plt.axhline(f2melf(traint.iloc[idx].f_min),c='w')
        plt.axhline(f2melf(traint.iloc[idx].f_max),c='w')

        plt.xlim(t_ctr-4,t_ctr+4)
        plt.show()

        del fig

In [ ]:
MEL_CUT = CNF['trainCUT'] # sec

_, n_mels, n_sample_per_minute = orig_meldata_tp.shape

cut_width = int(n_sample_per_minute*((MEL_CUT)/60)) 

meldata_tp = rfcx_cut_melspec(orig_meldata_tp,cut_width,traint.t_min.values,traint.t_max.values,traint.mymelid.values)
    
# del orig_meldata_tp

    
# 最大値が1になるように正規化 とりあえず全体のMIN,MAXで正規化 
g_meldata_tp_max = meldata_tp.max()
g_meldata_tp_min = meldata_tp.min()

meldata_tp = (meldata_tp-g_meldata_tp_min)/(g_meldata_tp_max-g_meldata_tp_min)


# for vgg
CNF['meldata_std']  = meldata_tp.std()
CNF['meldata_mean'] = meldata_tp.mean()


In [ ]:
print(meldata_tp.shape)
plt.hist(meldata_tp.flatten());

# Data Loader

In [ ]:
from rfcxutil import dup3nmlresizeTrans # 1 channel melspec -> 3 channel 
import cv2

trnTransform = A.Compose([  A.ShiftScaleRotate( shift_limit=1, scale_limit=0.1, rotate_limit=0, interpolation=1, border_mode=cv2.BORDER_WRAP, value=None, mask_value=None, shift_limit_x=0.5, shift_limit_y=0, always_apply=False, p=1) , # x よこ
dup3nmlresizeTrans()] )



valTransform = A.Compose([ dup3nmlresizeTrans() ] )


def get_loader(X,Y,batch_size=CNF['batch_size'],transform = None,shuffle=True, drop_last=True, mixup=False):

    ds = rfcx_SimpleDataset(X,Y,transform = transform)
    
    if mixup:
    
        def collate_mixup_fn(batch):

            if ds.Y is not None:

                Xs = torch.stack([x[0] for x in batch])
                Ys = torch.stack([x[1] for x in batch])
                
                Xmixup = ( (Xs[0]+Xs[1])/2 ).unsqueeze(0)
                Ymixup = ((Ys[0]+Ys[1]).clip(0,1) ).unsqueeze(0)
                
                Xs = torch.cat([Xs,Xmixup],axis=0)
                Ys = torch.cat([Ys,Ymixup],axis=0)

                return Xs, Ys

            else:

                Xs = torch.stack([x for x in batch])

                return Xs
            
        _collate_fn = collate_mixup_fn
        
    else:
        
        _collate_fn = ds.collate_fn
    
    loader = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=CNF['num_workers'], collate_fn=_collate_fn, drop_last=drop_last)
    loader.num = len(X)
    
    return loader



In [ ]:
# fold=0
# trn_idx,val_idx = traint.query(f"fold != {fold}").index.tolist(), traint.query(f"fold == {fold}").index.tolist()
    
# Xtrn,Xval = meldata_tp[trn_idx],meldata_tp[val_idx] # dfのindexでndarrayから抜き出してるからindexが0～Nって感じでちゃんと並んでる前提
# Ytrn,Yval = traint[LABEL_COLS].values[trn_idx],traint[LABEL_COLS].values[val_idx]
    
# trn_ld = get_loader(Xtrn,Ytrn,transform=trnTransform,shuffle=True,drop_last=True,mixup=CNF['mixup']) # trnTransform is defined outside this class

In [ ]:
# for x,y in trn_ld:
#     #print(x)
#     plt.imshow( x[0].numpy().transpose(1,2,0) )
#     break

# Model

In [ ]:
from rfcxutil import rfcx_myGlobalAvgPool2d,rfcx_myGlobalMaxPool2d # 横方向にavg_pool


class GroupFC(nn.Module):

    def __init__(self,n_mel,n_classes,feat_dim):
        super(GroupFC, self).__init__()
        self.n_mel = n_mel ; self.n_classes = n_classes ; self.feat_dim = feat_dim # maxpool前のfeat_dim
        self.dropout0 = nn.Dropout(p=0.2) # 0.0だとちょっとだけ悪くなる
        
        self.simplefc =  nn.Sequential(  nn.Linear( 2*self.feat_dim*self.n_mel, self.n_classes ) ) 
        
    def forward(self, inputs_avg, inputs_max):
        
        x_avg = torch.cat([ self.dropout0(inputs_avg[:,self.feat_dim*(_i):self.feat_dim*(_i+1)] ) for _i in range(self.n_mel) ], 1)
        x_max = torch.cat([ self.dropout0(inputs_max[:,self.feat_dim*(_i):self.feat_dim*(_i+1)] ) for _i in range(self.n_mel) ], 1)
        
        x = torch.cat([x_avg,x_max],axis=-1)
        
        return self.simplefc( x )
                                       

class VggModel(nn.Module):

    def __init__(self, CNF,n_classes=24):
        
        super(VggModel, self).__init__()
        
        self.n_mel = 4
        self.CNF = CNF
        self.n_classes = n_classes
        self.baseModel = torch.hub.load('pytorch/vision:v0.6.0','vgg11_bn' , pretrained=True if self.CNF['isTrain'] else False)
        self.baseModel.avgpool = rfcx_myGlobalAvgPool2d(n_mel=self.n_mel)
        self.baseModel.maxpool = rfcx_myGlobalMaxPool2d(n_mel=self.n_mel)
        
        self.baseModel.fc = GroupFC(n_mel=self.n_mel,n_classes=n_classes,feat_dim=512) 
        
    def vgg_unit(self,x):
        
        x = self.baseModel.features(x)
        x_avgpool = self.baseModel.avgpool(x)
        x_maxpool = self.baseModel.maxpool(x)
        
        x = self.baseModel.fc(x_avgpool,x_maxpool)
        
        return x
        
    def forward(self, x):
        
        xh  = ( x[:,:,0::2]+x[:,:,1::2] )/2 # 64mel -> batch x mel 
        
        return   ( self.vgg_unit(x) + self.vgg_unit(xh)  )/2
        

# Resnet

In [ ]:
class ResnetModel(nn.Module):

    def __init__(self, CNF,n_classes=24):
        
        super(ResnetModel, self).__init__()
        
        self.n_mel = 4
        self.CNF = CNF
        self.n_classes = n_classes
        self.baseModel = torch.hub.load('pytorch/vision:v0.6.0','resnet18' , pretrained=True if self.CNF['isTrain'] else False)
        
        self.baseModel.fc = nn.Linear( 512, self.n_classes )
        
    def forward(self, x):
        
        return self.baseModel(x)
        
    

# Fitter

In [ ]:
from rfcxutil import  SaveFeatures
    
    
class FitterNN():
    
    def __init__(self,CNF,fold_id,prefix="resnet"): # prefixは小文字で
        
        self.CNF = CNF ; self.prefix = prefix ; self.fold_id = fold_id
        self.model = VggModel(self.CNF) if 'vgg' in self.prefix else None
        self.model = ResnetModel(self.CNF) if 'resnet' in self.prefix else None
        
        self.final_layer = self.model.baseModel.layer4 if 'resnet' in self.prefix else None
        
        
    def fit(self,Xtrn,Xval,Ytrn,Yval,Wtrn,Wval):
        
        seed_everything(42)
        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        
        optimizer = optim.Adam( [{'params': self.model.parameters(), 'lr': self.CNF['lr']}] , lr=self.CNF['lr'] ) 
        
        scheduler = LambdaLR(optimizer, lr_lambda=lambda ep: 0.01+(1-0.01)*np.cos(0.5*np.pi*ep/self.CNF['epoch'])**2, verbose=True )
        
        trn_criterion, val_criterion = [ torch.nn.BCEWithLogitsLoss( pos_weight=torch.FloatTensor( w ).to(device) ) for w in [Wtrn,Wval] ]
        
        trn_ld = get_loader(Xtrn,Ytrn,transform=trnTransform,shuffle=True,drop_last=True,mixup=self.CNF['mixup']) # trnTransform is defined outside this class
        
        isSeqVal = Xval.shape[-1] != Xtrn.shape[-1] # 自動でseqvalかvalを設定
        print(f"isSeqVal:{isSeqVal}")
        if isSeqVal:
            Xval = self.split60sec2sequence(Xval)
            val_batch_size = int(np.ceil(60/self.CNF['trainCUT'])) # 60秒が何分割されるか
            Yval = np.repeat(Yval, val_batch_size, axis=0) # 無駄だけど、データローディング順を合わせるためだけ
            
        else:
            val_batch_size = CNF['batch_size']
            
        print(f"VAL_BATCH_SIZE:{val_batch_size}")
            
        val_ld = get_loader(Xval,Yval,batch_size=val_batch_size,transform=valTransform,shuffle=False,drop_last=False)
        
        self.model.to(device)
        ema = ModelEMA(self.model) if self.CNF['EMA'] else None
        trn_losses,val_losses,scores = [],[],[] ; best_score = -np.inf
            
        for epoch in range(self.CNF['epoch']):

            self.model.train() ;  tr_loss = 0
            for _X, _Y in trn_ld: 
                
                outputs = self.model(_X.to(device))
                loss = trn_criterion(outputs, _Y.to(device))

                loss.backward(); optimizer.step();optimizer.zero_grad()
                _ = ema.update(self.model) if self.CNF['EMA'] else None
                tr_loss+=loss.detach().cpu().numpy().sum()
            scheduler.step()
            tr_loss = tr_loss/len(Xtrn)

            
            #
            # seq pred の時は batchは1sequenceを表す
            #
            with torch.no_grad():

                self.model.eval() ; vl_loss = 0 ; gt = [] ; preds = []
                for _X, _Y in val_ld:

                    outputs = self.model(_X.to(device))
                    
                    if isSeqVal:
                        outputs = self.getPredictionFromSeqwisePred(outputs)
                        outputs,_ = torch.max(outputs,axis=1)
                        vl_loss+=val_criterion(outputs[0],_Y[0].to(device)) # 1 つのレコードに対するval_loss
                    
                    else:
                        vl_loss+=val_criterion(outputs,_Y.to(device)).sum()
                    
                    preds +=  [ torch.sigmoid(outputs).cpu().numpy() ]

                preds=np.vstack(preds) 
                vl_loss = vl_loss/len(Xval)
            
            
            if isSeqVal:
                _score_class, _weight = lwlrap(Yval[::val_batch_size,:],preds); score = (_score_class * _weight).sum()
            else:
                _score_class, _weight = lwlrap(Yval,preds); score = (_score_class * _weight).sum()

            if score > best_score:

                best_score = score 
                os.makedirs(f"{self.prefix}", exist_ok=True); torch.save(self.model.state_dict(), f"{self.prefix}/{self.prefix}_model_fold{self.fold_id}.pth")

            print(f"EP{epoch:02d}:TRN_LOSS {tr_loss:.5f} VAL_LOSS {vl_loss:.5f} SCORE {score:.5f}")
            trn_losses.append(tr_loss);val_losses.append(vl_loss);scores.append( score );torch.cuda.empty_cache()
        
        history = {'trn_loss':trn_losses,'val_loss':val_losses,'score':scores,'best_score':best_score}
        pickle.dump(history, open(f"{self.prefix}/{self.prefix}_history_fold{self.fold_id}.pkl", 'wb'))
        
        return history
        
    def predict(self,X,returnCam=False):
        
        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        self.model.to(device)
        self.model.load_state_dict(torch.load(f"{self.prefix}/{self.prefix}_model_fold{self.fold_id}.pth"))
        
        ld = get_loader(X,None,transform=valTransform,shuffle=False,drop_last=False)
        with torch.no_grad():

            self.model.eval() ; preds = [] ; cam = []
            for _X in ld:
                
                if returnCam:
                    activated_features = SaveFeatures(self.final_layer)  # 登録

                pred = torch.sigmoid(self.model(_X.to(device)))
                
                if returnCam:
                    
                    activated_features.remove() # 登録解除
                    # fc weight
                    weight_fc_params = list(self.model.baseModel.fc.parameters())
                    weight_fc = np.squeeze(weight_fc_params[0].data.cpu().numpy())
                    # top 1 class id
                    class_idx = torch.topk(pred, 1)[1].int().squeeze() # 最新predsのtop1_cls_id
                    
                    for _id in range(len(class_idx)):
                        
                        _w  = weight_fc[class_idx[_id],:] # fc重み 1x512
                        _ft = activated_features.features[_id,:,:,:] # seq_id,_bandのlast conv出力 512x7x7
                        
                        cam.append( _w.dot(_ft.reshape((512, 7*7 ))).reshape(7,7) ) # 1x512 @ 512x7x7 -> 7x7 
                        
                        
                preds +=  [ pred.cpu().numpy() ]        
                    

            preds=np.vstack(preds);torch.cuda.empty_cache()
            if returnCam:
                cam = np.stack(cam)
            
        if returnCam:
            return preds,cam
        else:
            return preds
        
        
    def predictSeq(self,X):
        
        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        self.model.to(device)
        self.model.load_state_dict(torch.load(f"{self.prefix}/{self.prefix}_model_fold{self.fold_id}.pth"))
        
        X = self.split60sec2sequence(X)
        val_batch_size = int(np.ceil(60/self.CNF['trainCUT'])) # 60秒が何分割されるか
            
        ld = get_loader(X,None,batch_size=val_batch_size,transform=valTransform,shuffle=False,drop_last=False)
            
        with torch.no_grad():

            self.model.eval() ; preds = [] ; cam = []
            for _X in ld:
                
                outputs = self.model(_X.to(device))
                outputs = self.getPredictionFromSeqwisePred(outputs)
                outputs,_ = torch.max(outputs,axis=1)
                
                preds +=  [ torch.sigmoid(outputs).cpu().numpy() ]
                    

            preds=np.vstack(preds);torch.cuda.empty_cache()
        
        return preds
    
    
    def split60sec2sequence(self,X):
        
        # 60秒のメルデータXを小分けにする
        n_seq = int(np.ceil(60/self.CNF['trainCUT'])) # 60秒が何分割されるか
        n_data_in_60sec = X.shape[-1] # 60秒melの幅
        n_data_per_seq = int(np.round( self.CNF['trainCUT']*(n_data_in_60sec/60) )) # CNF['trainCUT']秒はデータにしていくつか
        n_seq_start_idx = np.floor(np.linspace(0, n_data_in_60sec-1-n_data_per_seq, num=n_seq, endpoint=True)).astype('int') # 60秒をCNF['trainCUT']秒に分割したそれぞれの区間に対応するオリジナルからの切り取り開始位置

        return np.stack([ X[:,:,idx:(idx+n_data_per_seq)] for idx in n_seq_start_idx ] ,axis=1).reshape(-1,128,n_data_per_seq) # batch_id x seq_id x 128(mel) x 375 -> ( batch_id x seq_id ) x 128 x 375
        
    def getPredictionFromSeqwisePred(self,P):
        
        n_seq = int(np.ceil(60/self.CNF['trainCUT'])) # 60秒が何分割されるか
        return P.reshape((-1,n_seq,24))
    
        


# Prepare Pseudo

In [ ]:
# !cp -r ../input/pthrfcx1221/vgg .

In [ ]:
if CNF['pseudo']:
    
    sub_df = pd.read_csv("../input/rfcx-species-audio-detection/sample_submission.csv")
    meldata_tst = np.load(CNF['meltestPath']) ; 
    meldata_tst = meldata_tst[:,:,:-1] # CUT6できりが良くなるように端数を切り捨て（大雑把wwwww）
    print(meldata_tst.shape)
    meldata_tst = (meldata_tst-g_meldata_tp_min)/(g_meldata_tp_max-g_meldata_tp_min) # 正規化 # plt.hist(meldata_tst.flatten());
    
    

In [ ]:
if CNF['pseudo']:
    
    fitter =  FitterNN(CNF, fold_id=0, prefix="vgg")
    seg_preds = []
    for i in range(3750//375):
        seg_preds.append( fitter.predict(meldata_tst[:,:,(375*i):(375*(i+1))]) )
        
    seg_preds = np.stack(seg_preds,axis=1)
    
    tstPred_segid_spid_confidence = np.array([ [ np.where(seg_pred==np.max(seg_pred))[0][0], np.where(seg_pred==np.max(seg_pred))[1][0], np.max(seg_pred) ] for seg_pred in seg_preds ])
    
    is_pseudo = tstPred_segid_spid_confidence[:,2]>0.5 # segid0,spid1,confidence2
    
    Xp = np.array([ mel_seg[:,(375*seg_id):(375*(seg_id+1))] for mel_seg, seg_id in zip(meldata_tst[is_pseudo], tstPred_segid_spid_confidence[:,0][is_pseudo].astype('int')) ])
    Yp = np.eye(24)[[ a for a in tstPred_segid_spid_confidence[:,1][is_pseudo].astype('int') ]]
    
    del meldata_tst
    gc.collect()


# Train

In [ ]:
if CNF['isTrain']:
    
    def getWeight(df):
        _tmp = 1/df.species_id.value_counts().sort_index().values/len(df)
        return 24*_tmp/sum(_tmp)

    #fold = 0
    
    for fold in CNF['targetFold']:
    
        trn_idx = traint.query(f"fold != {fold}").index.tolist() #; val_idx = traint.query(f"fold == {fold}").index.tolist()
        Xtrn = meldata_tp[trn_idx] # dfのindexでndarrayから抜き出してるからindexが0～Nって感じでちゃんと並んでる前提
        Ytrn = traint[LABEL_COLS].values[trn_idx]
        
        if CNF['validateSequence']:
            
            val_idx = trainu.query(f"fold == {fold}").index.tolist() # trainu に注意
        
            Xval = orig_meldata_tp[val_idx]  
            Yval = trainu.loc[val_idx,LABEL_COLS].values
            Wtrn,Wval = getWeight(traint[traint.fold!=fold]), getWeight(trainu[trainu.fold==fold])
            
        else:
            
            val_idx = traint.query(f"fold == {fold}").index.tolist() 
            
            Xval = meldata_tp[val_idx]
            Yval = traint.loc[val_idx,LABEL_COLS].values
            Wtrn,Wval = getWeight(traint[traint.fold!=fold]), getWeight(traint[traint.fold==fold])

        fitter =  FitterNN(CNF, fold, prefix="resnet")

        if CNF['pseudo']:
            Xtrn = np.vstack([Xtrn,Xp])
            Ytrn = np.vstack([Ytrn,Yp])
            #del Xp,Yp

        history = fitter.fit(Xtrn,Xval,Ytrn,Yval,Wtrn,Wval)

        plt.figure(figsize=(12,4))
        plt.subplot(121);plt.plot(history['trn_loss'],label='trn');plt.plot(history['val_loss'],label='val');plt.legend()
        plt.subplot(122);plt.plot(history['score'],label='score');plt.legend()
        plt.show();
        
        
        
        del Xtrn,Xval,Ytrn,Yval,Wtrn,Wval,fitter
        gc.collect()
        torch.cuda.empty_cache()

        print(f"FOLD{fold}:{history['best_score']}")

- CUT6 MultiLabel FOLD0:0.793 安定する
- CUT4 FOLD0:0.7904
- CUT6 FOLD0:0.7976
- CUT8 FOLD0:0.7882

# Check Validation for Seq

In [ ]:
if False:
    
    fold = 0
    fitter =  FitterNN(CNF, fold_id=fold, prefix="resnet")

    val_idx = trainu.query(f"fold == {fold}").index.tolist() # trainu に注意

    Xval = orig_meldata_tp[val_idx]  
    Yval = trainu.loc[val_idx,LABEL_COLS].values


    Pval = fitter.predictSeq(Xval)

    _score_class, _weight = lwlrap(Yval,Pval); score = (_score_class * _weight).sum()

    print(f"FOLD{fold}:{score:.4f}")

    del fitter

In [ ]:
if CNF['isTrain']==False:
    !cp -r {CNF['loadPthPath']} .

# CAM

In [ ]:
if CNF['plotCAM']:
    
    fold = 0
    val_idx = traint.query(f"fold == {fold}").index.tolist()

    fitter =  FitterNN(CNF, fold_id=fold, prefix="resnet")
    _Pval,cam = fitter.predict(meldata_tp[val_idx],returnCam=True) ; del fitter
    _score_class, _weight = lwlrap(traint.loc[val_idx,LABEL_COLS].values,_Pval); score = (_score_class * _weight).sum()
    print(f"FOLD{fold}:{score}")

    # _save(cam,f"cam_fold{fold}.pkl")
    # cam = _load('../input/cam1227fold0/cam_fold0.pkl')

    for cnt,idx in enumerate(val_idx):

        plt.figure(figsize=(15, 5))
        plt.imshow(meldata_tp[val_idx[cnt]][::-1],cmap='magma');
        plt.imshow(skimage.transform.resize(cam[cnt][::-1], (meldata_tp.shape[1], meldata_tp.shape[2])), alpha=0.25, cmap='jet');

        t_min = traint.iloc[idx].t_min
        t_max = traint.iloc[idx].t_max
        f_min = f2melf(traint.iloc[idx].f_min);f_max = f2melf(traint.iloc[idx].f_max)
        t_ctr = (t_min+t_max)/2
        t_start = t_ctr - CNF['trainCUT']/2
        im_t_min = int( (t_min - t_start)*meldata_tp.shape[-1]/CNF['trainCUT'] )
        im_t_max = int( (t_max - t_start)*meldata_tp.shape[-1]/CNF['trainCUT'] )

        im_f_min = meldata_tp.shape[-2] - f_min*meldata_tp.shape[-2]/16000
        im_f_max = meldata_tp.shape[-2] - f_max*meldata_tp.shape[-2]/16000

        plt.axvline(im_t_min,c='w'); plt.axvline(im_t_max,c='w')
        plt.axhline(im_f_max,c='w'); plt.axhline(im_f_min,c='w')

        plt.show();plt.close()


In [ ]:
# import seaborn as sns

# for cnt,idx in enumerate(val_idx):

#     plt.figure(figsize=(10,10))
#     cmap = sns.diverging_palette(220, 10, as_cmap=True)
#     sns.heatmap(valPredSeq[cnt].T,  cmap=cmap, vmin=0,vmax=1, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .3})

#     true_ids = list(np.where(trainu.loc[idx,LABEL_COLS]==1)[0])
#     true_ids_str = " ".join([ str(i) for i in true_ids])
    
#     pred_ids = np.argsort(np.max(valPredSeq[cnt],axis=0))[::-1][:5]
#     pred_ids_str = " ".join([ str(i) for i in pred_ids])

#     plt.title(f"{true_ids_str}\nPred {pred_ids_str}",color="red" if pred_ids[0]!=true_ids[0] else "blue")
    
#     plt.show()
    
#     plt.close()

# TEST

In [ ]:
del orig_meldata_tp,meldata_tp; gc.collect()

In [ ]:
sub_df = pd.read_csv("../input/rfcx-species-audio-detection/sample_submission.csv")


meldata_tst = np.load(CNF['meltestPath'])
print(meldata_tst.shape)


In [ ]:
# 最大値が1になるように正規化 とりあえずtrain_tp全体のMIN,MAXで正規化 個々に正規化するから多分意味ない
meldata_tst = (meldata_tst-g_meldata_tp_min)/(g_meldata_tp_max-g_meldata_tp_min)
plt.hist(meldata_tst.flatten());

In [ ]:
# plt.figure(figsize=(16,4))
# plt.subplot(121);plt.hist(meldata_tp.flatten(),label='trn');plt.title('trn');
# plt.subplot(122);plt.hist(meldata_tst.flatten(),label='tst');plt.title('tst');

In [ ]:
Xtst = meldata_tst

foldPred = []
for fold in CNF['targetFold']:
    
    fitter =  FitterNN(CNF, fold_id=fold, prefix="resnet")
    foldPred.append( fitter.predictSeq(Xtst) )

    del fitter

In [ ]:
tstPred_avg = np.mean(np.stack(foldPred),axis=0)
tstPred_max = np.max(np.stack(foldPred),axis=0)
_save(tstPred_avg,'tstPred_avg.pkl')
_save(tstPred_max,'tstPred_max.pkl')

## fold max

In [ ]:
sub_df[[ f"s{i}" for i in range(24)]] = tstPred_max
sub_df.to_csv("submission_max.csv", index=False)
sub_df.describe()

In [ ]:
plt.bar(x=np.arange(24),height=sub_df[LABEL_COLS].mean())

## fold average

In [ ]:
sub_df[[ f"s{i}" for i in range(24)]] = tstPred_avg
sub_df.to_csv("submission_avg.csv", index=False)
sub_df.describe()

In [ ]:
plt.bar(x=np.arange(24),height=sub_df[LABEL_COLS].mean())